In [1]:
import os

import warnings
warnings.filterwarnings('ignore')

from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage

from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader, PyMuPDFLoader, PyPDFLoader
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')


### 1. Data Ingestion

In [28]:
sample_documents = [
    Document(
        page_content="""
        Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.
        """,
        metadata={"source": "AI Introduction", "page": 1, "topic": "AI"},
    ),
    Document(
        page_content="""
        Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.
        """,
        metadata={"source": "ML Basics", "page": 1, "topic": "ML"},
    ),
    Document(
        page_content="""
        Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning has revolutionized computer vision, NLP, and speech recognition.
        """,
        metadata={"source": "Deep Learning", "page": 1, "topic": "DL"},
    ),
    Document(
        page_content="""
        Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
        Applications include chatbots, translation, sentiment analysis, and text summarization.
        """,
        metadata={"source": "NLP Overview", "page": 1, "topic": "NLP"},
    ),
]

sample_documents

[Document(metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}, page_content='\n        Artificial Intelligence (AI) is the simulation of human intelligence in machines.\n        These systems are designed to think like humans and mimic their actions.\n        AI can be categorized into narrow AI and general AI.\n        '),
 Document(metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='\n        Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.\n        '),
 Document(metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='\n        Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revo

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=[" "]
)

chunks = text_splitter.split_documents(sample_documents)

In [4]:
chunks

[Document(metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}, page_content='Artificial Intelligence (AI) is the simulation of human intelligence in machines.\n        These systems are designed to think like humans and mimic their actions.\n        AI can be categorized into narrow AI and general AI.'),
 Document(metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'),
 Document(metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recogn

### 2. Load the embedding model

In [5]:
embedding = OpenAIEmbeddings(model='text-embedding-3-large', dimensions=1536)

sample_text = "What is machin elearning"
sample_embedding = embedding.embed_query(sample_text)
len(sample_embedding)

1536

In [6]:
texts = ["AI", "ML", 'DL', 'NN']

batch_embeddings = embedding.embed_documents(texts)
len(batch_embeddings)

4

### Compare embeddings woth cosine similarity

In [7]:
import numpy as np

def compare_embeddings(text1: str, text2: str):
    emb1 = np.array(embedding.embed_query(text1))
    emb2 = np.array(embedding.embed_query(text2))

    similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))

    return similarity

In [8]:
compare_embeddings("INDIA", "CHINA")

np.float64(0.5854221448239765)

In [9]:
compare_embeddings("INDIA", "KENIA")

np.float64(0.4195690088445148)

In [10]:
compare_embeddings("ML", "DL")

np.float64(0.5379085787893086)

### Create FAISS Vector Store

In [11]:
vector_store = FAISS.from_documents(
    documents=chunks,
    embedding=embedding
)

print(f"vector store created with {vector_store.index.ntotal} vectors")

vector store created with 4 vectors


### Save the vectors in FAISS

In [12]:
vector_store.save_local(folder_path="FAISS_index")
print("vectore store saved at 'FAISS_index'")

vectore store saved at 'FAISS_index'


In [13]:
loaded_vector_store = FAISS.load_local(folder_path="FAISS_index", 
                                       embeddings=embedding, 
                                       allow_dangerous_deserialization=True)

In [14]:
# let's perform similarity search

loaded_vector_store.similarity_search("what is deep learning")

[Document(id='9fb1b03e-4e91-4a84-9d8f-6e2692141fac', metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recognition.'),
 Document(id='a0ea7af0-18b2-4336-847d-1887a8bca18d', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'),
 Document(id='67018fdd-7c4e-4fa5-9ba5-23e607326947', metadata={'source': 'NLP Overview', 'page': 1, 'topic': 'NLP'}, page_content='Natural Language Processing (NLP) is a branch of AI that helps computers understand human la

In [15]:
# let's perform similarity score

loaded_vector_store.similarity_search_with_score("what is deep learning", k=2)

[(Document(id='9fb1b03e-4e91-4a84-9d8f-6e2692141fac', metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recognition.'),
  np.float32(0.63186)),
 (Document(id='a0ea7af0-18b2-4336-847d-1887a8bca18d', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'),
  np.float32(1.0620883))]

In [16]:
# let's perform similarity with metadata filtering

filter = {"topic": "ML"}

filtered_result = loaded_vector_store.similarity_search(
    query="what is machine learning",
    k=2,
    filter=filter
    )
filtered_result

[Document(id='a0ea7af0-18b2-4336-847d-1887a8bca18d', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.')]

### Building RAG chain with LCEL

In [17]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

# os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

llm = ChatOpenAI(model="chatgpt-4o-latest")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000016C85E42390>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000016C85E431D0>, root_client=<openai.OpenAI object at 0x0000016C85E42090>, root_async_client=<openai.AsyncOpenAI object at 0x0000016C85E42F10>, model_name='chatgpt-4o-latest', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [18]:
simple_prompt = ChatPromptTemplate.from_template("""
                                                 Answer the question based only on the following context:
                                                 Context: {context}
                                                 Question: {Question}
                                                 Answer:""")



In [19]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":3})
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000016C85E13F50>, search_kwargs={'k': 3})

In [20]:
from typing import List

def format_doc(docs: List[Document])-> str:
    formatted = []
    for i, doc in enumerate(docs):
        source = doc.metadata.get("source", "Unknown")
        formatted.append(f"Document {i+1} (Source: {source})\n{doc.page_content}")
    
    print(formatted)
    return "\n\n".join(formatted)


In [21]:
from langchain.output_parsers import StructuredOutputParser

rag = ({"Context": retriever | format_doc, "Question": RunnablePassthrough()}
       | simple_prompt
       | llm
       | StrOutputParser()
       )
rag

{
  Context: VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000016C85E13F50>, search_kwargs={'k': 3})
           | RunnableLambda(format_doc),
  Question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['Question', 'context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['Question', 'context'], input_types={}, partial_variables={}, template='\n                                                 Answer the question based only on the following context:\n                                                 Context: {context}\n                                                 Question: {Question}\n                                                 Answer:'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000016C85E42390>, async_client=<openai.resources.chat.completions.c

### Conversational RAG Chain

In [23]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt with memory placeholder
conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the provided context to answer questions."),
    MessagesPlaceholder(variable_name="chat_history"),  # will be filled by memory
    ("human", "Context: {context}\n\nQuestion: {input}")
])

# Initialize memory
memory = ConversationBufferMemory(return_messages=True)

# Define chain without RunnableWithHistory
def conversational_rag(user_input: str):
    # 1. Load history
    chat_history = memory.load_memory_variables({}).get("history", [])

    # 2. Add context via retriever
    context = format_doc(retriever.invoke(user_input))

    # 3. Run through prompt → llm → parser
    response = (
        RunnablePassthrough.assign(context=lambda x: context, input=lambda x: user_input, chat_history=lambda x: chat_history)
        | conversational_prompt
        | llm
        | StrOutputParser()
    ).invoke({})

    # 4. Save turn in memory
    memory.save_context({"input": user_input}, {"output": response})

    return response


C:\Users\BhagwatChate\AppData\Local\Temp\ipykernel_1352\3904111289.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [25]:
print(conversational_rag("What is AI?"))
print(conversational_rag("What is the capital of India?"))
print(conversational_rag("What questions did I ask before?"))


['Document 1 (Source: AI Introduction)\nArtificial Intelligence (AI) is the simulation of human intelligence in machines.\n        These systems are designed to think like humans and mimic their actions.\n        AI can be categorized into narrow AI and general AI.', 'Document 2 (Source: ML Basics)\nMachine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.', 'Document 3 (Source: NLP Overview)\nNatural Language Processing (NLP) is a branch of AI that helps computers understand human language.\n        It combines computational linguistics with machine learning and deep learning models.\n        Applications include chatbots, translation, sentiment analysis, and text summarization.']
AI, or Artificial Intelligence, is the simulation of human intelligence in machines. These systems are designed to think 

### Streaming RAG Chain

In [ ]:
from operator import itemgetter
llm = ChatOpenAI(model="chatgpt-4o-latest")

streaming_rag_chain = (
    {
        "context": itemgetter("Question") | retriever | format_doc,
        "Question": itemgetter("Question"),
    }
    | simple_prompt
    | llm
    | StrOutputParser()
)

response = streaming_rag_chain.invoke({"Question": "What is AI?"})
print(response)


In [ ]:
streaming_rag_chain.invoke({"Question": "What is AI?"})

### RAG Chain Types:
- simple RAG chain - basic Q&A
- Conversational RAG chain - maintains chat history
- Streaming RAG chain - supports token streaming